<font size=6>Ocean Data Analysis - Analyzing Sea Surface Temperature w/ Variational Inference</font>

Interesting Research
https://www.nature.com/articles/s41467-018-08066-0

In [52]:
import warnings
warnings.filterwarnings('ignore')

import xarray as xr
!pip install netcdf4
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

import sklearn 
import sklearn.mixture as mix 
import scipy.stats as stats 

import seaborn as sns
sns.set()

In [53]:
# Texas Gulf of Mexico
ds = xr.open_dataset('https://dods.ndbc.noaa.gov/thredds/dodsC/data/stdmet/44078/44078.ncml')
ds = ds.sel(time=slice('2003-04-01','2023-04-25'))

# Get buoy data from NOAA
noaa_df = ds.to_dataframe().reset_index()

In [54]:
# Get sea surface temps
cov = noaa_df[['sea_surface_temperature']]
cov = cov.dropna().reset_index().drop(columns=['index'])

# Fit mixture model 
num_components = 4
dpgmm_model = mix.BayesianGaussianMixture(
    n_components=num_components, 
    weight_concentration_prior_type='dirichlet_process',
    n_init=2,
    max_iter=50)
p = dpgmm_model.fit_predict(cov)

# Count States
state_counts = np.zeros(num_components)
for M in p:
  state_counts[M] += 1 
print(state_counts)

[13048. 14060. 13293. 12096.]


In [ ]:
# Plot States
fig2, ax2=plt.subplots()

for M in range(len(p)): 
  if p[M] == p[-1]:
    ax2.axvline(M, color='black', alpha=0.002) 
    
sns.lineplot(data=cov['sea_surface_temperature'].values, ax=ax2, alpha=0.6)
plt.ylabel('sst')
plt.xlabel('time')
plt.show()

In [ ]:
# Plot States
fig,ax=plt.subplots()
covm = [] 
for M in range(len(p)): 
  if p[M] == p[-1]:
    try:
        covm.append(cov['sea_surface_temperature'].values[M])
    except:
        print('skip')
    
sns.lineplot(data=covm, ax=ax, alpha=0.9)
plt.ylabel('sst')
plt.xlabel('time')
plt.show()

print(np.mean(covm))